In [1]:
import sys
sys.path.append('/glade/u/home/dkimpara/')
sys.path.append('../')

import pbs_utils.qsubPython as qpy

### resource recs:
- rap: 1h ncpus=24:mem=500GB
- gfs: 1h ncpus=6:mem=50GB
- hrrr: 30min ncpus=32 mem=550GB
    -  for hrrr - only run on one case study *day* at a time (or less)

resource syntax: select=1:ncpus=12:mem=100GB
walltime syntax: 00:00:00

In [2]:
job = qpy.qsubPython('NAML0001', 'dkimpara@ucar.edu', 'casper')
job.append_job_script(['module load conda',
                       'conda activate ptype',
                       'cd /glade/u/home/dkimpara/ptype-physical/composite-soundings',
                       'mkdir -p outfiles'])

In [3]:
print(job)

qsubPython(account='NAML0001', user_list='dkimpara@ucar.edu', queue='casper', mail_option='a', jobname=None, walltime='01:00:00', resources=None, outfile='out.out', errorfile='out.out', job_script=['export TMPDIR=/glade/scratch/$USER/temp', 'mkdir -p $TMPDIR', 'source /etc/profile.d/modules.sh', 'module load conda', 'conda activate ptype', 'cd /glade/u/home/dkimpara/ptype-physical/composite-soundings', 'mkdir -p outfiles'])


In [4]:
funcs = [
         ("xmr.compute_by_disagree", 'disagree'), 
         ("partial(xmr.compute_by_conf, [0.3, 0.5])", 'conf35'),
         ("partial(xmr.compute_by_conf, [0.7, 0.9])", 'conf79')
        ]


## Submit jobs for rap and gfs

In [5]:
#-i 0 for kentucky, 1 for new_york_1, 2 for new_york_2
model = 'rap' #or rap
job.set_job_attrs(resources='select=1:ncpus=18:mem=1500GB',
                  walltime='00:30:00')
for func, funcname in funcs:    
    for i in range(1,3):
        jobname = f'{model}_{funcname}_{i}'
        job.set_job_attrs(jobname=jobname,
                          outfile=f'outfiles/{jobname}.out',
                          errorfile= f'outfiles/{jobname}.out')
        
        job.submit([f'''python -u /glade/u/home/dkimpara/ptype-physical/applications/map_reduce_run.py -i {i} -m {model} -o {jobname} -f "{func}"'''])

7922928.casper-pbs
7922929.casper-pbs
7922930.casper-pbs


## submit jobs for hrrr data

In [6]:
base_dir = '/glade/campaign/cisl/aiml/ptype/ptype_case_studies/'
dirs = ['kentucky/hrrr/20220223',
        'kentucky/hrrr/20220224',
        'new_york_1/hrrr/20221215',
        'new_york_1/hrrr/20221216',
        'new_york_1/hrrr/20221217',
        'new_york_2/hrrr/20220203',
        'new_york_2/hrrr/20220204'
        ]
model = 'hrrr'

In [7]:
for dir in dirs[1:]:
    for func, funcname in funcs:    
        jobname = f'{model}_{funcname}_{dir[:1] + dir[-2:]}'
        job.set_job_attrs(jobname=jobname,
                        outfile=f'outfiles/{jobname}.out',
                        errorfile= f'outfiles/{jobname}.out')
        
        job.submit([f'''python -u /glade/u/home/dkimpara/ptype-physical/applications/map_reduce_run.py -d {dir} -m {model} -o {jobname} -f "{func}"'''])

7922931.casper-pbs
7922932.casper-pbs
7922933.casper-pbs
